In [10]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraryies
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [11]:
weekly_nfl = nfl.import_weekly_data([2023])


Downcasting floats.


In [12]:
# Create a DF for WR's and TE's
wr_df_two = weekly_nfl[(weekly_nfl["position"] == "WR") | (weekly_nfl["position"] == "TE")]
new_columns = ['player_display_name','fantasy_points', 'fantasy_points_ppr','receiving_yards','receiving_yards_after_catch','receptions','targets','receiving_tds','receiving_air_yards','receiving_first_downs','receiving_epa','target_share','air_yards_share','racr','wopr','headshot_url']
wr_df = wr_df_two[new_columns]
wr_df= wr_df[wr_df['fantasy_points_ppr'] != 0]
wr_df.count()
#wr_df_two.count()

player_display_name            513
fantasy_points                 513
fantasy_points_ppr             513
receiving_yards                513
receiving_yards_after_catch    513
receptions                     513
targets                        513
receiving_tds                  513
receiving_air_yards            513
receiving_first_downs          513
receiving_epa                  503
target_share                   503
air_yards_share                503
racr                           503
wopr                           503
headshot_url                   509
dtype: int64

In [4]:
#create df for average WR
wr_average_df = wr_df.groupby('player_display_name').mean().reset_index()
wr_average_df.set_index('player_display_name', inplace=True)


In [5]:
# remove nans and infinites
wr_average_df.fillna(wr_average_df.mean(), inplace=True)
wr_average_df.dropna(inplace=True)
wr_average_df.replace([np.inf, -np.inf], np.nan, inplace=True)


In [6]:
# create elbow curve
# Overall Individual Seseason
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(wr_average_df)
    inertia.append(km.inertia_)
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="NFL WR Average Elbow Curve", xticks=k)  

:Curve   [k]   (inertia)

In [7]:
# PCA Elbow Curve
pca = PCA(n_components=5)

# Get two principal components for the iris data.
pca = pca.fit_transform(wr_average_df)
fin_pca_df = pd.DataFrame(
    data = pca, columns=["pc1","pc2","pc3","pc4","pc5"]
)
fin_pca_df

inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(fin_pca_df)
    inertia.append(km.inertia_)
    
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [8]:
# OVR MODEL
def get_clusters(k, data):
    # Create a copy of the DataFrame
    data = data.copy()

    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Fit the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["class"] = model.labels_

    return data

In [9]:
#create ml of 5 clusters
wr_five_df = get_clusters(5, wr_average_df)
wr_five_df

,fantasy_points,fantasy_points_ppr,receiving_yards,receiving_yards_after_catch,receptions,targets,receiving_tds,receiving_air_yards,receiving_first_downs,receiving_epa,target_share,air_yards_share,racr,wopr,class
player_display_name,,,,,,,,,,,,,,,
A.J. Brown,7.966667,14.633333,79.666664,34.000000,6.666667,10.000000,0.000000,115.666664,4.333333,2.675799,0.335354,0.412893,0.690635,0.792056,0
Adam Thielen,11.700000,18.366667,70.333336,22.333334,6.666667,8.333333,0.666667,68.666664,4.666667,5.355588,0.200151,0.204866,1.099771,0.443633,0
Adam Trautman,3.400000,8.400000,34.000000,16.000000,5.000000,5.000000,0.000000,18.000000,1.000000,2.053385,0.156250,0.136364,1.888889,0.329830,3
Alec Pierce,2.533333,4.533333,25.333334,3.333333,2.000000,4.000000,0.000000,68.666664,1.333333,0.118307,0.102702,0.242975,0.570582,0.324136,2
Allen Lazard,3.600000,5.933333,36.000000,8.666667,2.333333,4.333333,0.000000,62.000000,1.666667,1.728422,0.186931,0.403045,0.712214,0.562528,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xavier Hutchinson,0.900000,1.900000,9.000000,2.000000,1.000000,1.000000,0.000000,7.000000,1.000000,0.105245,0.023256,0.026923,1.285714,0.053730,1
Zach Ertz,2.766667,7.433333,27.666666,10.666667,4.666667,6.666667,0.000000,42.666668,1.000000,-1.183802,0.234811,0.200234,0.377252,0.492380,3
Zach Pascal,0.900000,1.900000,9.000000,12.000000,1.000000,1.000000,0.000000,-3.000000,0.000000,0.617539,0.034483,-0.012766,0.000000,0.042788,1


In [71]:
# df of all classes
sorted_class = wr_five_df.sort_values(["class"])
class_one = sorted_class[(sorted_class["class"] == 0)]
class_two = sorted_class[(sorted_class["class"] == 1)]
class_three = sorted_class[(sorted_class["class"] == 2)]
class_four = sorted_class[(sorted_class["class"] == 3)]
class_five = sorted_class[(sorted_class["class"] == 4)]

In [72]:
# Create ML for traditional stats
fantasy_points = wr_average_df.groupby("player_display_name").mean()['fantasy_points']
fantasy_points_ppr = wr_average_df.groupby("player_display_name").mean()['fantasy_points_ppr']
yards = wr_average_df.groupby("player_display_name").mean()['receiving_yards']
receptions = wr_average_df.groupby("player_display_name").mean()['receptions']
targets = wr_average_df.groupby("player_display_name").mean()['targets']
tds = wr_average_df.groupby("player_display_name").mean()['receiving_tds']
target_share = wr_average_df.groupby("player_display_name").mean()['target_share']

trad_ml_df = pd.DataFrame({
    "fantasy_points": fantasy_points,
    "fantasy_points_ppr": fantasy_points_ppr,
    "yards": yards,
    "receptions": receptions,
    "targets": targets,
    "tds":tds,
    "target_share": target_share
})

trad_ml_df.head()

,fantasy_points,fantasy_points_ppr,yards,receptions,targets,tds,target_share
player_display_name,,,,,,,
A.J. Brown,5.40,10.90,54.0,5.5,8.0,0.0,0.303030
Adam Thielen,7.30,11.80,33.0,4.5,5.5,0.5,0.177419
Adam Trautman,3.40,8.40,34.0,5.0,5.0,0.0,0.156250
Alec Pierce,1.65,3.15,16.5,1.5,2.5,0.0,0.070720
Allen Lazard,3.45,5.45,34.5,2.0,4.0,0.0,0.191111


In [73]:
# create ml trad 5 k


wr_trad_df = get_clusters(5, trad_ml_df)
wr_trad_df

,fantasy_points,fantasy_points_ppr,yards,receptions,targets,tds,target_share,class
player_display_name,,,,,,,,
A.J. Brown,5.40,10.90,54.0,5.5,8.0,0.0,0.303030,4
Adam Thielen,7.30,11.80,33.0,4.5,5.5,0.5,0.177419,0
Adam Trautman,3.40,8.40,34.0,5.0,5.0,0.0,0.156250,0
Alec Pierce,1.65,3.15,16.5,1.5,2.5,0.0,0.070720,2
Allen Lazard,3.45,5.45,34.5,2.0,4.0,0.0,0.191111,0
...,...,...,...,...,...,...,...,...
Xavier Hutchinson,0.90,1.90,9.0,1.0,1.0,0.0,0.023256,2
Zach Ertz,3.85,9.85,38.5,6.0,9.0,0.0,0.304598,0
Zach Pascal,0.90,1.90,9.0,1.0,1.0,0.0,0.034483,2


In [56]:
# df of all classes
trad_class = wr_trad_df.sort_values(["class"])
class_one_trad = trad_class[(trad_class["class"] == 0)]
class_two_trad = trad_class[(trad_class["class"] == 1)]
class_three_trad = trad_class[(trad_class["class"] == 2)]
class_four_trad = trad_class[(trad_class["class"] == 3)]
class_five_trad = trad_class[(trad_class["class"] == 4)]

In [61]:
#make xlsx with ml group

wr_trad_df.to_excel('wr_ml_trad.xlsx', index=True)